In [1]:
cd ../langchain-ChatGLM-master/

F:\AI\hohoChat\langchain-ChatGLM-master


In [2]:
import os
import time

import torch.cuda
import torch.backends

from configs import model_config
from models.chatglm_llm import ChatGLM
from chains.local_doc_qa import *
from textsplitter import ChineseTextSplitter
from utils import torch_gc

from langchain.text_splitter import MarkdownTextSplitter, CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import UnstructuredMarkdownLoader 

In [3]:
DATA_DIR = "H:\AI\data\法律\法律法规文件\Laws-master"

In [4]:
def time_str_YmdHmS():
    current_time = time.time()
    local_time = time.localtime(current_time)
    time_str = time.strftime('%Y%m%d%H%m%S', local_time)
    return time_str

In [5]:
def get_filepaths_at_path(item_path):
    if os.path.isfile(item_path):
        return [item_path]
    
    result_list = []
    for item in os.listdir(item_path):
        path = os.path.join(item_path, item)
        file_paths = get_filepaths_at_path(path)
        result_list.extend(file_paths)
    
    return result_list

In [6]:
file_paths = get_filepaths_at_path(DATA_DIR)
file_paths = [file_path for file_path in file_paths if os.path.basename(file_path) != '_index.md']

In [7]:
len(file_paths)

3084

In [8]:
# text_splitter = ChineseTextSplitter(pdf = False)
text_splitter = MarkdownTextSplitter(chunk_size = 500, chunk_overlap = 100)

docs = []
for file_path in file_paths:
    loader = UnstructuredMarkdownLoader(file_path)
    docs += loader.load_and_split(text_splitter)

In [9]:
len(docs)

44115

In [10]:
embeddings = HuggingFaceEmbeddings(model_name = model_config.embedding_model_dict["ernie-base"], 
                                   model_kwargs = {'device': model_config.EMBEDDING_DEVICE})

No sentence-transformers model found with name C:\Users\Administrator/.cache\torch\sentence_transformers\nghuyong_ernie-3.0-base-zh. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\Administrator/.cache\torch\sentence_transformers\nghuyong_ernie-3.0-base-zh were not used when initializing ErnieModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing ErnieModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ErnieModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
vector_store = FAISS.from_documents(docs, embeddings)

In [12]:
vs_path = f"../outputs/vector_store/law_FAISS_{time_str_YmdHmS()}"
vector_store.save_local(vs_path)

In [13]:
FAISS.similarity_search_with_score_by_vector = similarity_search_with_score_by_vector
vector_store.chunk_size = model_config.CHUNK_SIZE

In [14]:
llm = ChatGLM()
llm.load_model(model_name_or_path = model_config.llm_model_dict[model_config.LLM_MODEL],
               llm_device = model_config.LLM_DEVICE,
               use_ptuning_v2 = model_config.USE_PTUNING_V2)
llm.history_len = LLM_HISTORY_LEN

D:\ProgramData\Anaconda3\envs\hoho_chatgpt\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Administrator\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been co

No compiled kernel found.
Compiling kernels : C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kernels_parallel.c -shared -o C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kernels_parallel.so
Compile default cpu kernel failed, using default cpu kernel code.
Compiling gcc -O3 -fPIC -std=c99 C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kernels.c -shared -o C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a

In [19]:
def answer_based_on_knowledge(query, vector_store, llm, history = []):
    related_docs_with_score = vector_store.similarity_search_with_score(query, k = VECTOR_SEARCH_TOP_K)
    related_docs = get_docs_with_score(related_docs_with_score)
    prompt = generate_prompt(related_docs, query)

    # if streaming:
    #     for result, history in self.llm._stream_call(prompt = prompt,history = chat_history):
    #         history[-1][0] = query
    #         response = {"query": query,
    #                     "result": result,
    #                     "source_documents": related_docs}
    #         yield response, history
    # else:
    for result, history in llm._call(prompt = prompt, history = history, streaming = False):
        history[-1][0] = query
        response = {"query": query,
                    "result": result,
                    "source_documents": related_docs}
        yield response, history

In [20]:
from IPython.display import display, Markdown, clear_output

def display_answer(query, history = []):
    for resp, history in answer_based_on_knowledge(query, vector_store, llm, history):
        clear_output(wait = True)
        display(Markdown(resp['result']))
    
    return resp, history

In [17]:
question = "信用卡欠款不还会遭到什么处罚？"
answer, history = display_answer(question, history = [])

根据已知信息，信用卡欠款不还可能会遭到以下处罚：

1. 利率优惠和信用额度降低：如果信用卡欠款不还，银行可能会降低信用卡的利率，并减少信用额度，以提醒用户及时还款。

2. 法律处罚：根据刑法，信用卡欠款不还可能会被视为犯罪行为，会受到法律处罚。例如，可能会被指控信用卡欺诈罪或欠款罪，并面临刑事指控和法律诉讼。

3. 信用受损：信用卡欠款不还可能会影响用户的信用，导致用户的信用评分下降，影响其申请贷款和其他金融服务。

4. 银行起诉：如果信用卡欠款不还，银行可能会向法院起诉用户，要求其还款。如果用户无法还款，银行可能会申请法院执行，将用户的财产划拨到银行，以完成还款。

信用卡欠款不还可能会面临多种处罚，包括利率优惠和信用额度降低、法律处罚、信用受损和银行起诉。

In [22]:
question = "一名员工对女同事实施了性骚扰行为，女同事向公司进行举报，但公司却没有采取必要的措施来制止这种行为。\n\n公司未采取必要措施预防和制止性骚扰，导致女同事的权益受到侵害，该公司是否需要承担责任？"
answer, history = display_answer(question, history = [])

根据已知信息，无法回答该问题。没有提供足够的相关信息，无法判断公司是否采取了必要的措施预防和制止性骚扰行为，因此无法判断公司是否需要承担责任。

In [23]:
question = "遭遇强制拆迁 如何用法律保护自己?"
answer, history = display_answer(question, history = [])

根据已知信息，可以回答该问题。

强制拆迁可能会对被拆迁人的生活造成严重的影响，因此，如何用法律保护自己是一个关键问题。

根据《中华人民共和国宪法》和《中华人民共和国土地管理法》的规定，被拆迁人有权依法维护自己的权益。在遭遇强制拆迁时，被拆迁人可以通过以下方式保护自己：

1. 申请法律援助：被拆迁人可以向当地法律援助中心申请法律援助，获得法律咨询和代理服务，帮助自己依法维权。

2. 依法维权：被拆迁人可以通过法律途径维护自己的权益，例如向当地法院提起诉讼、申请仲裁等。在诉讼中，被拆迁人需要提供证据，证明房屋属于自己的财产，并依法维护自己的权益。

3. 申请补偿：被拆迁人需要依法申请补偿，并了解当地的补偿标准。如果补偿标准不合理，被拆迁人可以通过法律途径维护自己的权益。

4. 与拆迁部门沟通：被拆迁人需要与拆迁部门进行沟通，了解拆迁计划和拆迁程序，并协商解决相关问题。如果被拆迁人不同意拆迁计划，可以通过法律途径维护自己的权益。

5. 寻求社会支持：被拆迁人可以通过社会支持，获得社会舆论的支持，帮助自己依法维权。

在遭遇强制拆迁时，被拆迁人需要依法维护自己的权益，通过法律途径解决问题。

In [24]:
question = "口头录取通知有没有法律效力？"
answer, history = display_answer(question, history = [])

根据已知信息，可以回答该问题。口头录取通知的法律效力取决于具体的语境和录取程序。在某些情况下，口头录取通知可以具有法律效力，但在其他情况下可能不起作用。例如，在某些学校或教育机构中，可能需要通过书面文件或电子邮件确认录取通知的效力。因此，需要根据具体情况来确定口头录取通知的法律效力。

In [25]:
start_time = time.time()

vs_path = "../outputs/vector_store/law_FAISS_20230518230554"
vector_store2 = FAISS.load_local(vs_path, embeddings)

print(f"loading seconds: {time.time() - start_time} s")

loading seconds: 0.5334064960479736 s
